In [ ]:
import os
import zipfile
from IPython.display import FileLink

def zip_directory(directory=".", file_name="directory.zip"):
    """
    Zips all files in a directory.

    Parameters:
    directory (str): The directory to be zipped. Default is the current working directory.
    file_name (str): The name of the zipped file (including .zip). Default is 'directory.zip'.

    Returns:
    FileLink: A hyperlink to download the zip file.
    """
    os.chdir(directory)
    zip_ref = zipfile.ZipFile(file_name, mode='w')
    for folder, _, files in os.walk(directory):
        for file in files:
            if file_name in file:
                pass
            else:
                zip_ref.write(os.path.join(folder, file))
    zip_ref.close()
    return FileLink(file_name)

# Example usage:
# Assuming your output files are in '/kaggle/working/'
# You can call the function like this in a notebook cell:
zip_directory(directory="/kaggle/working/", file_name="my_output.zip")